In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
from pathlib import Path
from PIL import Image
import requests
from tqdm import tqdm
import os
import random
from transformers import AutoProcessor, LlavaForConditionalGeneration, AutoTokenizer, CLIPModel
import torch
import os
import re 
from tqdm import tqdm

In [2]:
print("GPU available:", torch.cuda.is_available())
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

model = LlavaForConditionalGeneration.from_pretrained("llava-hf/llava-1.5-7b-hf", torch_dtype=torch.float16)
model.to(device)
processor = AutoProcessor.from_pretrained("llava-hf/llava-1.5-7b-hf")

GPU available: True


/home/longju/anaconda3/envs/llava/lib/python3.10/site-packages/transformers/models/llava/configuration_llava.py:100: FutureWarning: The `vocab_size` argument is deprecated and will be removed in v4.42, since it can be inferred from the `text_config`. Passing this argument has no effect
  warnings.warn(


Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [3]:
def generate_response_llava_wImage(text_prompt, image_source, device):
    image = Image.open(image_source)
    prompt = text_prompt
    
    inputs = processor(text=prompt, images=image, return_tensors="pt")

    inputs = {k: v.to(device) for k, v in inputs.items()}

    generate_ids = model.generate(**inputs, max_length=100000)

    full_response = processor.batch_decode(generate_ids, skip_special_tokens=True, clean_up_tokenization_spaces=False)[0]


    response_parts = full_response.split("ASSISTANT:", 1)
    response = response_parts[1].strip() if len(response_parts) > 1 else ""
    return response

In [4]:
### TEST example

image_source = ""

prompt_sum = (f"<image>\n"
              f"USER: Generate a comprehensive summary based on the image, to describe the contents of the image and the culture related knowledge about this picture. Limit response to 3 sentences. \nASSISTANT:")

res = generate_response_llava_wImage(prompt_sum, image_source, device)
print(res)

The image features a Christmas tree with a Mickey Mouse and Minnie Mouse dressed as Santa Claus and Mrs. Claus, surrounded by presents. The scene is set in a festive atmosphere, likely at a theme park or a special event. This image represents the popular culture of the United States, particularly the holiday season, and the iconic characters of Mickey and Minnie Mouse.


In [5]:
base_path = ''

# Load DataFrame
file_path = ''
df = pd.read_csv(file_path)

In [6]:
def apply_captioning(row):
    full_image_path = os.path.join(base_path, row['imageRelPath'])
    prompt = (f"<image>\n"
              f"USER: Generate a comprehensive summary based on the image, to describe the contents of the image and the culture related knowledge about this picture. Limit response to 3 sentences. \nASSISTANT:")
    return generate_response_llava_wImage(prompt, full_image_path, device)

df['llava7b_caption'] = [apply_captioning(row) for row in tqdm(df.to_dict('records'), desc="Generating Captions")]

output_file_path = ''
df.to_csv(output_file_path, index=False)

print("Captions generated and saved successfully.")

Generating Captions: 100%|██████████| 6464/6464 [4:46:46<00:00,  2.66s/it]   


Captions generated and saved successfully.
